In [131]:
# --- 1. Import des bibliothèques nécessaires ---
import pandas as pd
import os
import numpy as np

# --- 2. Définition du chemin et chargement des données ---
# Remplacez par votre chemin d'accès local
DATA_PATH = r"C:/Users/Etudiant/Documents/Cours M2 IDMC/Bussiness Intelligent BI/TPPOWERBIAFRIQUE"
file_path = os.path.join(DATA_PATH, "Annex_FR_Tab4.xlsx")

# Vérification de l'existence du fichier
if not os.path.exists(file_path):
    print(f"ERREUR : Le fichier '{file_path}' n'a pas été trouvé.")
    # Arrête le script si le fichier n'est pas là
    exit()
else:
    print(f"Fichier trouvé : {file_path}")

# Chargement de la feuille 'Tab4' du fichier Excel
try:
    df = pd.read_excel(file_path, sheet_name='Tab4', header=1) # On suppose que la ligne 1 est l'en-tête
    print("Données chargées avec succès.")
except Exception as e:
    print(f"Une erreur est survenue lors du chargement : {e}")
    exit()

# --- 3. Nettoyage et préparation des données ---

# Renommage des colonnes pour une utilisation facile
# Note : Les noms sont basés sur la structure de l'image (12 colonnes financières)
df.columns = [
    'Index', 'Code', 'Pays', 
    'Recettes_2013', 'Depenses_2013', 'Solde_2013',
    'Recettes_2014', 'Depenses_2014', 'Solde_2014',
    'Recettes_2015', 'Depenses_2015', 'Solde_2015',
    'Recettes_2016', 'Depenses_2016', 'Solde_2016'
]

# Suppression des lignes non pertinentes (lignes vides, totaux, etc.)
# On supprime les lignes où le 'Code' pays est manquant, c'est un bon identifiant.
df.dropna(subset=['Code'], inplace=True)
df.reset_index(drop=True, inplace=True)

print(f"Le jeu de données nettoyé contient {df.shape[0]} pays et {df.shape[1]} colonnes.")

# Liste des colonnes numériques à convertir
numeric_cols = [
    'Recettes_2013', 'Depenses_2013', 'Solde_2013',
    'Recettes_2014', 'Depenses_2014', 'Solde_2014',
    'Recettes_2015', 'Depenses_2015', 'Solde_2015',
    'Recettes_2016', 'Depenses_2016', 'Solde_2016'
]

# Conversion en numérique, les erreurs deviennent NaN (Not a Number)
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
print("Conversion des colonnes numériques effectuée.")


# --- 4. Analyse des Valeurs Manquantes ---
print("\n" + "="*50)
print("--- 4. ANALYSE DES VALEURS MANQUANTES ---")
print("="*50)

# Compter les valeurs manquantes par colonne
missing_values = df.isnull().sum()
print("\nNombre de valeurs manquantes par colonne :")
print(missing_values[missing_values > 0]) # Affiche seulement les colonnes avec des manquants

# Identifier les pays avec des données manquantes
missing_data_rows = df[df.isnull().any(axis=1)]
print("\nPays concernés par les données manquantes :")
print(missing_data_rows[['Pays'] + numeric_cols])


# --- 5. Analyse des Valeurs Aberrantes (Outliers) ---
print("\n" + "="*50)
print("--- 5. ANALYSE DES VALEURS ABERRANTES (MÉTHODE IQR) ---")
print("="*50)

# Boucle sur chaque colonne numérique pour détecter les outliers
for col in numeric_cols:
    # Retirer les NaN pour les calculs de quartiles
    data_col = df[col].dropna()
    
    # Calcul des quartiles et de l'IQR
    Q1 = data_col.quantile(0.25)
    Q3 = data_col.quantile(0.75)
    IQR = Q3 - Q1
    
    # Définition des bornes pour les outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Identification des outliers
    outliers_mask = (df[col] < lower_bound) | (df[col] > upper_bound)
    outliers_df = df.loc[outliers_mask]
    
    if not outliers_df.empty:
        print(f"\n--- Outliers détectés pour '{col}' ---")
        print(f"(Bornes : < {lower_bound:.2f} ou > {upper_bound:.2f})")
        print(f"Nombre d'outliers : {len(outliers_df)}")
        print("Détails :")
        print(outliers_df[['Pays', col]].round(2))
    else:
        print(f"\n--- Aucun outlier détecté pour '{col}' ---")

# --- 6. Aperçu Final ---
print("\n" + "="*50)
print("--- 6. APERÇU FINAL DU DATAFRAME PRÊT À L'ANALYSE ---")
print("="*50)
print(df.head())

Fichier trouvé : C:/Users/Etudiant/Documents/Cours M2 IDMC/Bussiness Intelligent BI/TPPOWERBIAFRIQUE\Annex_FR_Tab4.xlsx
Données chargées avec succès.
Le jeu de données nettoyé contient 54 pays et 15 colonnes.
Conversion des colonnes numériques effectuée.

--- 4. ANALYSE DES VALEURS MANQUANTES ---

Nombre de valeurs manquantes par colonne :
Recettes_2013    2
Depenses_2013    2
Solde_2013       1
Recettes_2014    2
Depenses_2014    2
Solde_2014       1
Recettes_2015    2
Depenses_2015    3
Solde_2015       1
Recettes_2016    2
Depenses_2016    2
Solde_2016       1
dtype: int64

Pays concernés par les données manquantes :
             Pays  Recettes_2013  Depenses_2013  Solde_2013  Recettes_2014  \
34     Mozambique           32.2           35.0        -2.9           31.4   
43        Somalie            NaN            NaN         NaN            NaN   
45  Soudan du Sud            NaN            NaN       -12.8            NaN   

    Depenses_2014  Solde_2014  Recettes_2015  Depenses_2015